# Confluence + LLM = QA

In [1]:
%pip install pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

In [2]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_host = environ.get('CONFLUENCE_HOST')
confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')
gigachat_pro_token = environ.get('GIGACHAT_PRO_TOKEN')
yandex_token = environ.get('YC_API_KEY')
openchat_host = environ.get('OPENCHAT_HOST')

## Датасет

### Выгрузка документов с Confluence

In [29]:
from atlassian import Confluence
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

# text_splitter = SentenceTransformersTokenTextSplitter(model_name="nizamovtimur/rubert-tiny2-wikiutmn")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4096,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

def get_document_content_by_id(confluence: Confluence, page_id: str) -> tuple[str | None, str | None]:
    page = confluence.get_page_by_id(page_id, expand="space,body.export_view")
    page_link = page["_links"]["base"] + page["_links"]["webui"]
    page_body = page["body"]["export_view"]["value"]
    page_download = (
        page["_links"]["base"] + page["_links"]["download"]
        if "download" in page["_links"].keys()
        else ""
    )
    try:
        if len(page_body) > 50:
            page_body = page["body"]["export_view"]["value"]
            soup = BeautifulSoup(page_body, "html.parser")
            page_body_text = soup.get_text(separator=" ")
            page_content = page_body_text.replace(" \n ", "")
        elif ".pdf" in page_download.lower():
            loader = PyPDFLoader(page_download.split("?")[0])
            page_content = " ".join(
                [page.page_content for page in loader.load_and_split()]
            )
        else:
            return None, None
    except Exception as e:
        print(e)
        return None, None
    return page_content, page_link


confluence = Confluence(url=confluence_host, token=confluence_token)
page_ids = []
count_start = 0
limit = 100
while True:
    query = f"space = study order by id"
    pages = confluence.cql(query, start=count_start, limit=limit)["results"]
    if len(pages) == 0:
        break
    page_ids += [
        page["content"]["id"] for page in pages if "content" in page.keys()
    ]
    count_start += limit
confluence_documents = []
for page_id in page_ids:
    children = confluence.cql(f"parent={page_id}")["results"]
    if len(children) > 0:
        continue
    page_content, page_link = get_document_content_by_id(confluence, page_id)
    if page_content is None:
        continue
    confluence_documents.append(
        Document(page_content=page_content, metadata={"page_link": page_link})
    )

confluence_documents = text_splitter.split_documents(confluence_documents)
len(confluence_documents)

124

### Генерация вопросов с помощью LLM

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = """
Сделай глубокий вдох и действуй как студент. Составь 3 разнообразных вопроса к документу в тройных кавычках. Используй разговорный стиль речи и студенческую лексику. Отвечать на вопросы не нужно, напиши каждый вопрос с новой строки.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [19]:
from langchain_openai import ChatOpenAI

openchat = ChatOpenAI(
    # model="openchat_3.5",
    model="saiga_llama3_8b",
    openai_api_key="EMPTY",
    openai_api_base=openchat_host,
    temperature=0.6,
)

openchat_chain = prompt | openchat
openchat_chain.invoke({"content": confluence_documents[0].page_content}).content

'"Как я могу сделать справку-вызов самостоятельно и какие шаги мне нужно выполнить для этого?"\n\n"Что мне нужно делать, если я имею академическую задолженность за предыдущую сессию?"\n\n"Можно ли мне оформить справку-вызов на текущую сессию, если я обучаюсь заочной формой и сдал предыдущую сессию не успешно?"'

In [ ]:
openchat_docs = []
for doc in confluence_documents:
    query = {"content": doc.page_content}
    chat_questions = openchat_chain.invoke(query).content.strip().split("\n")
    print(chat_questions)
    for q in chat_questions:
        q = q.strip()
        if len(q) < 5:
            continue
        openchat_docs.append({
            "question": q,
            "document": doc.page_content
        })
openchat_docs = pd.DataFrame(openchat_docs)
openchat_docs

In [29]:
openchat_docs_mini = openchat_docs[openchat_docs.document.apply(len) < 3500]
openchat_docs_mini

,question,document
0,Как я могу сформировать справку-вызов самостоя...,Сформировать справку-вызов Вы можете самостоят...
1,"Сколько времени у меня есть до начала сессии, ...",Сформировать справку-вызов Вы можете самостоят...
2,"Если я не уложил работы за прошлую сессию, мог...",Сформировать справку-вызов Вы можете самостоят...
3,"""Какой адрес и номер кабинета нужно указать дл...",По вопросу получения справки для военкомата Ва...
4,Какие терминалы доступны для оформления справк...,Студенты очной формы обучения оформляют справк...
...,...,...
332,Что является основанием для принятия решения К...,"деятельности в соответствии с критериями, ука..."
333,Что происходит с оставшимися заявлениями обуча...,"деятельности в соответствии с критериями, ука..."
334,Что такое Школа перспективных исследований (SA...,Институт экологической и сельскохозяйственной ...
335,Какие факультеты находятся в Институте социаль...,Институт экологической и сельскохозяйственной ...


In [32]:
openchat_docs_mini.to_csv("openchat_docs_mini.csv")

### Загрузка датасета с Hugging Face

In [8]:
%pip install datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, DatasetDict

dataset_qda = load_dataset("nizamovtimur/wikiutmn-study-gigachat")
if isinstance(dataset_qda, DatasetDict):
    train_dataset_qda = dataset_qda['train'].to_pandas()
    test_dataset_qda = dataset_qda['test'].to_pandas()
else:
    train_dataset_qda = pd.DataFrame()
    test_dataset_qda = pd.DataFrame()
len(train_dataset_qda), len(test_dataset_qda)

d:\git\virtassist\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(355, 67)

In [4]:
train_dataset_qda

,question,document,human_answer
0,Где продлять студак?,Для восстановления студенческого билета Вам не...,
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер...",
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...,
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи...",
...,...,...,...
350,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ...",
351,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ...",
352,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон...",
353,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон...",


In [5]:
test_dataset_qda

,question,document,human_answer
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Информация, о том, как поменять электив, отпра..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...


In [6]:
test_dataset_qda_docs_len = test_dataset_qda.document.apply(len)
np.mean(test_dataset_qda_docs_len), np.std(test_dataset_qda_docs_len), np.min(test_dataset_qda_docs_len), np.max(test_dataset_qda_docs_len)

(827.7014925373135, 729.8501178155583, 134, 2685)

### Загрузка датасета из файла

In [17]:
test_dataset_qda = pd.read_csv("test_gigachat_docs.csv", index_col=0)
test_dataset_qda.head(5)

,question,document
0,Как можно сформировать справку-вызов самостоят...,Сформировать справку - вызов Вы можете самосто...
1,Куда следует направлять подписанную скан-копию...,Сформировать справку - вызов Вы можете самосто...
2,Где можно получить справку для военкомата?,По вопросу получения справки для военкомата Ва...
4,Какой номер телефона у Отдела мобилизационной ...,По вопросу получения справки для военкомата Ва...
6,Какие места на территории Университета можно и...,Студенты очной формы обучения оформляют справк...


## Векторный индекс документов

### Структура БД

In [ ]:
!docker build ../db -t db

In [8]:
!docker run --name db --rm -p 5432:5432 --mount source=db_volume,target=/var/lib/postgresql/data --env-file ../.env -d db

a8e8dc4d58730f9fbc28d30ae6e0e4a98819d7c6b9f2af086940d7b82ddf413d


In [6]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, create_engine, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [7]:
class Base(DeclarativeBase):
    pass

class Chunk(Base):
    __tablename__ = "chunk"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text())
    sbert_312: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    sbert_768: Mapped[Optional[Vector]] = mapped_column(Vector(768))
    sbert_1024: Mapped[Optional[Vector]] = mapped_column(Vector(1024))
    gigachat_embeddings: Mapped[Optional[Vector]] = mapped_column(Vector(1024))

In [11]:
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.execute(text('DROP TABLE chunk'))
    session.commit()
Base.metadata.create_all(engine)

### Заполнение документами

In [8]:
documents = pd.concat([train_dataset_qda["document"], test_dataset_qda["document"]]).drop_duplicates().reset_index(drop=True)
len(documents)

42

In [13]:
with Session(engine) as session:
    for document in documents:
        doc = Chunk(
            # text=document.page_content,
            text=document,
        )
        session.add(doc)
    session.commit()

### Вычисление векторных представлений

#### RuSBERT-Tiny

In [42]:
from sentence_transformers import SentenceTransformer

# rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")
rusbert_model = SentenceTransformer('saved_models/rubert-tiny2-wikiutmn', device="cpu")
len(rusbert_model.encode("мама мыла раму"))

d:\git\virtassist\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


312

In [44]:
%%timeit
rusbert_model.encode(openchat_docs_mini.question[0])

7.71 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [82]:
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      chunk.sbert_312 = rusbert_model.encode(chunk.text)
      session.add(chunk)
      session.flush()
   session.commit()

In [86]:
def answer_sbert_312(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.sbert_312.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

answer_sbert_312("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

#### mpnet

In [51]:
from sentence_transformers import SentenceTransformer

# mpnet = SentenceTransformer('saved_models/sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device="cuda")
mpnet = SentenceTransformer('saved_models/paraphrase-multilingual-mpnet-base-v2-wikiutmn', device="cpu")
len(mpnet.encode("мама мыла раму"))

768

In [52]:
%%timeit
mpnet.encode(openchat_docs_mini.question[0])

65.7 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      chunk.sbert_768 = mpnet.encode(chunk.text)
      session.add(chunk)
      session.flush()
   session.commit()

In [54]:
def answer_sbert_768(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.sbert_768.cosine_distance(
                            mpnet.encode(question)
                            )).limit(1)).first().text

answer_sbert_768("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.\xa0 Для успешной аттестации по дисциплине «Физическая культура:

#### e5-large

In [10]:
from sentence_transformers import SentenceTransformer

# e5_model = SentenceTransformer('saved_models/intfloat/multilingual-e5-large', device="cuda")
e5_model = SentenceTransformer('saved_models/multilingual-e5-large-wikiutmn', device="cpu")
len(e5_model.encode("мама мыла раму"))

1024

In [48]:
%%timeit
e5_model.encode(openchat_docs_mini.question[0])

215 ms ± 4.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      chunk.sbert_1024 = e5_model.encode(chunk.text)
      session.add(chunk)
      session.flush()
   session.commit()

In [12]:
def answer_sbert_1024(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.sbert_1024.cosine_distance(
                            e5_model.encode(question)
                            )).limit(1)).first().text

answer_sbert_1024("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

#### GigaChatEmbeddings

In [14]:
from langchain_community.embeddings import GigaChatEmbeddings

gigachat_embeddings = GigaChatEmbeddings(credentials=gigachat_token, verify_ssl_certs=False)
result = gigachat_embeddings.embed_documents(texts=["мама мыла раму"])
print(len(result[0]))

1024


In [56]:
%%timeit
gigachat_embeddings.embed_documents(texts=[openchat_docs_mini.question[0]])

623 ms ± 8.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      try:
         chunk.gigachat_embeddings = gigachat_embeddings.embed_documents(texts=[chunk.text])[0]
      except:
         print(chunk.text[:100])
         chunk.gigachat_embeddings = None
      session.add(chunk)
      session.flush()
   session.commit()

Заявления о переводе принимаются два раза в год, как правило : для студентов очной и очно [UNK] заоч
1. Подать заявление о переводе можно лично, обратившись в Единый деканат, ул. Семакова, 18, холл 3 э
Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модуле
Пакет документов для подачи заявления на перевод : 1. Заявление о зачислении переводом https : / / w
Документ подписан простой электронной подписью Информация о владельце : ФИО : Романчук Иван Сергееви
директора, заместителя директора института школы, филиала, начальника ОУДКС, начальника у правления 
##ционной комиссии. Последующие восстановления производятся на место по договору об оказании платных
При равном числе голосов председатель аттестационной комиссии имеет право решающего голоса. Аттестац
Документ подписан простой электронной подписью Информация о владельце : ФИО : Романчук Иван Сергееви
. 1. 20. Дисциплины ( иные виды учебных работ ), с оставляющие академическую разницу и ( ил

In [16]:
def answer_gigachat_embeddings(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.gigachat_embeddings.cosine_distance(
                            gigachat_embeddings.embed_documents(texts=[question])[0]
                            )).limit(1)).first().text

answer_gigachat_embeddings("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

In [17]:
banned_by_gigachat = []
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      if chunk.gigachat_embeddings is None:
         banned_by_gigachat.append(chunk)

len(banned_by_gigachat)

18

### Выбор нужного фрагмента через векторный индекс

In [19]:
# test_dataset_qda["sbert_312"] = test_dataset_qda["question"].apply(answer_sbert_312)
# test_dataset_qda["sbert_768"] = test_dataset_qda["question"].apply(answer_sbert_768)
# test_dataset_qda["sbert_1024"] = test_dataset_qda["question"].apply(answer_sbert_1024)
test_dataset_qda["gigachat_embeddings"] = test_dataset_qda["question"].apply(answer_gigachat_embeddings)
test_dataset_qda.head(5)

,question,document,human_answer,gigachat_embeddings
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр...",Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр...",Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...,"Прежде, чем выбрать элективы, рекомендуем почи..."
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...,Подать заявление на отчисление переводом можно...


### Выбор лучшего алгоритма

#### Accuracy

In [21]:
for column in test_dataset_qda.columns[2:]:
    print(column, sum(test_dataset_qda[column].apply(lambda x: "" if x is None else x) == test_dataset_qda.document) / len(test_dataset_qda.document))

human_answer 0.31343283582089554
gigachat_embeddings 0.5373134328358209


#### ROUGE-L

In [53]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [22]:
from rouge import Rouge
rouge = Rouge()

for column in test_dataset_qda.columns[2:]:
    print(column, rouge.get_scores(test_dataset_qda[column].apply(lambda x: "-" if x is None else x), test_dataset_qda["document"], avg=True)['rouge-l'])

human_answer {'r': 0.6170558241367031, 'p': 0.9958196833933283, 'f': 0.7133136761037521}
gigachat_embeddings {'r': 0.6138765422480673, 'p': 0.6498791978849756, 'f': 0.6188192588396773}


## SBERT Fine Tuning


 * https://www.sbert.net/docs/training/overview.html
 * https://huggingface.co/blog/how-to-train-sentence-transformers

### Тонкая настройка

In [62]:
import math
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

train_set = []
for index, row in train_dataset_qda.iterrows():
    train_set.append(InputExample(texts=[row['question'], row['document']]))

finetuned_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(finetuned_model)
# train_loss = losses.MegaBatchMarginLoss(finetuned_model)

num_epochs = 10
warmup_steps = math.ceil(len(train_set) * num_epochs * 0.1)

finetuned_model.fit(train_objectives=[(train_dataloader, train_loss)],
                    epochs=num_epochs,
                    warmup_steps=warmup_steps,
                    output_path="saved_models/rubert-tiny2-wikiutmn")


d:\git\virtassist\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

: 

### Save to Hub

In [19]:
finetuned_model = SentenceTransformer('saved_models/multilingual-e5-large-wikiutmn', device="cpu")
finetuned_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
finetuned_model.save_to_hub(repo_id="nizamovtimur/multilingual-e5-large-wikiutmn", token=hf_write_token)

## Большие языковые модели


https://python.langchain.com/docs/use_cases/question_answering/

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = """Действуйте как Вопрошалыч — виртуальный помощник студента ТюмГУ.
Используйте следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента.
Оставьте ссылки, адреса и телефоны как есть. Если ответа в тексте нет, напишите "ответ не найден".
Предоставьте краткий, точный и полезный ответ, иначе вас заменят на GPT-5.

\"\"\"
{context}
\"\"\"

Вопрос студента: {question}"""

prompt = PromptTemplate.from_template(prompt_template)

In [37]:
# prompt_template_with_history = """Действуй как Вопрошалыч — виртуальный помощник студента ТюмГУ.
# Кратко ответь на вопрос студента по предоставленным текстам.
# Не придумывай и не изменяй ссылки, адреса и телефоны. Если ответа нет, напиши "ответ не найден".

# \"\"\"
# {prev_context}

# {context}
# \"\"\"

# Вопрос студента: {prev_question}
# Ответ: {prev_answer}
# Вопрос студента: {question}
# Ответ:
# """

# prompt_with_history = PromptTemplate.from_template(prompt_template)

In [16]:
from langchain_core.messages import AIMessage

def get_answer(chain, question, context=None):
        if context is None:
                # context = answer_sbert_312(question)
                # context = answer_sbert_768(question)
                context = answer_sbert_1024(question)
        query = {"context": context,
                 "question": question}
        answer = chain.invoke(query)
        if isinstance(answer, AIMessage):
                answer = answer.content
        return answer.strip()

### Метрика: BERTScore

In [26]:
%pip install bert-score --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="ru")

d:\git\virtassist\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### GigaChat

In [18]:
from langchain.llms import GigaChat
giga = GigaChat(model="GigaChat-preview", temperature=1.222, credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
get_answer(giga_chain, "К кому обратиться по вопросам стипендии?")

'Ответ не найден.'

In [13]:
%%timeit
get_answer(giga_chain, "К кому обратиться по вопросам стипендии?")

679 ms ± 9.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### GigaChat Pro

In [30]:
from langchain.llms import GigaChat
giga_pro = GigaChat(model="GigaChat-Pro", credentials=gigachat_token, verify_ssl_certs=False)
giga_pro_chain = prompt | giga_pro
get_answer(giga_pro_chain, "К кому обратиться по вопросам стипендии?")

'Для получения справки о доходах, размере стипендии или пособии вам следует обратиться в Сервисный центр бухгалтерии. Это можно сделать, написав на почту 12222@utmn.ru или позвонив по номеру (3452) 59-76-76. Центр находится по адресу: г. Тюмень, ул. Кирова, д. 25/1. Часы работы: с понедельника по четверг с 9:00 до 17:00, в пятницу с 9:00 до 16:00. Перерыв на обед с 12:30 до 13:15. Суббота и воскресенье - выходные дни.'

In [33]:
%%timeit
get_answer(giga_pro_chain, "К кому обратиться по вопросам стипендии?")

9.03 s ± 2.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### YandexGPT

пока изучал, 149 руб 03 коп улетело...

In [17]:
%pip install --upgrade --quiet  yandex_chain

Note: you may need to restart the kernel to use updated packages.


In [34]:
from yandex_chain import YandexLLM

yagpt = YandexLLM(folder_id="b1gvhukmt6vde5nnr73s", api_key=str(yandex_token), use_lite=False)
yagpt_chain = prompt | yagpt
get_answer(yagpt_chain, "К кому обратиться по вопросам стипендии?")

'По вопросам, связанным со стипендией, вам необходимо обратиться в Сервисный центр бухгалтерии. Вы можете написать на почту 12222 @ utmn. ru или позвонить по телефону (3452) 59-76-76.'

In [35]:
%%timeit
get_answer(yagpt_chain, "К кому обратиться по вопросам стипендии?")

The slowest run took 4.28 times longer than the fastest. This could mean that an intermediate result is being cached.
1.52 s ± 808 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `OpenChat-3.5-0106`, `saiga_llama3_8b`

 * https://github.com/imoneoi/openchat
 * https://huggingface.co/IlyaGusev/saiga_llama3_8b
 * https://huggingface.co/Vikhrmodels/Vikhr-7B-instruct_0.4

запущена на NVIDIA A100-PCIE-40GB

In [27]:
%pip install langchain_openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_openai import ChatOpenAI

openchat = ChatOpenAI(
    model="openchat_3.5",
    # model="saiga_llama3_8b",
    openai_api_key="EMPTY",
    openai_api_base=openchat_host,
    temperature=0.6,
)

openchat_chain = prompt | openchat
get_answer(openchat_chain, "К кому обратиться по вопросам стипендии?")

'Обратитесь в Сервисный центр бухгалтерии по адресу г. Тюмень, ул. Кирова, д. 25/1, или вы можете написать на почту 12222@utmn.ru или позвонить по телефону (3452) 59-76-76. Часы работы: пн-чт 09.00-17.00, пт 09.00-16.00, обед 12.30-13.15, сб,вс - выходные дни.'

In [91]:
%%timeit
get_answer(openchat_chain, "К кому обратиться по вопросам стипендии?")

699 ms ± 262 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 1. Тест генерации по заранее найденному документу

#### GigaChat

In [19]:
gigachat_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question'], row['document']))
gigachat_answers = pd.Series(gigachat_answers)
gigachat_answers.head(5)

0    Для восстановления магнитной карты обратитесь ...
1                                     Ответ не найден.
2    Для восстановления проходки необходимо подать ...
3                                     Ответ не найден.
4    В течение 5 (пяти) рабочих дней со дня издания...
dtype: object

In [20]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.551 Recall: 0.587 F1 score: 0.566'

#### GigaChat Pro

In [17]:
gigachat_pro_answers = []
for index, row in test_dataset_qda[14:].iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question'], row['document']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)
gigachat_pro_answers.head(5)

0    Для подачи заявления на перевод в ТюмГУ из дру...
1    Об элективах можно узнать на сервисе «Отзывус»...
2    Заявление на выход из академического отпуска п...
3                                     Ответ не найден.
4    Если вам не пришла стипендия, вам следует обра...
dtype: object

In [35]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_pro_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.786 Recall: 0.789 F1 score: 0.785'

#### YandexGPT 3

тут возник такой мем: `ai.textGenerationCompletionRequestsPerHour.rate rate quota limit exceed: allowed 100 requests`

хаха, лан, пошёл гулять в магаз

In [46]:
yagpt_answers = []
for index, row in test_dataset_qda.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question'], row['document']))
yagpt_answers = pd.Series(yagpt_answers)
yagpt_answers.head(5)

0    Согласно тексту, для перевода в ТюмГУ необходи...
1    Обратитесь к заключению врачебной комиссии. Ес...
2    В представленном тексте есть информация только...
3    Для того чтобы получить отпуск по уходу за реб...
4    Заявление о выходе из академического отпуска п...
dtype: object

In [68]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(yagpt_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.792 Recall: 0.794 F1 score: 0.790'

#### `OpenChat-3.5-0106`, `saiga_llama3_8b`

In [39]:
openchat_answers = []
for index, row in test_dataset_qda.iterrows():
    openchat_answers.append(get_answer(openchat_chain, row['question'], row['document']))
openchat_answers = pd.Series(openchat_answers)
openchat_answers.head(5)

0    Для восстановления магнитной карты необходимо ...
1    "Заявления на восстановление в Университет по ...
2    "Подать заявление на ее восстановление в Едино...
3    Для успешной аттестации по дисциплине «Физичес...
4    В течение 5 рабочих дней со дня издания приказ...
dtype: object

In [32]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(openchat_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.693 Recall: 0.755 F1 score: 0.720'

In [42]:
saved_test_dataset_qda = pd.read_csv("test_dataset_qda.csv", index_col=0)
saved_test_dataset_qda["saiga_llama3_8b"] = openchat_answers
saved_test_dataset_qda.to_csv("test_dataset_qda.csv")

### 2. Тест всего пайплайна

#### GigaChat

In [21]:
gigachat_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question']))
gigachat_answers = pd.Series(gigachat_answers)
gigachat_answers.head(5)

0    Обратитесь в Единый деканат для подачи заявлен...
1                                      Ответ не найден
2    Для восстановления проходки необходимо подать ...
3                                     Ответ не найден.
4                                     Ответ не найден.
dtype: object

In [22]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.496 Recall: 0.541 F1 score: 0.516'

#### GigaChat Pro

In [31]:
gigachat_pro_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)
gigachat_pro_answers.head(5)

0    Для восстановления магнитной карты (пропуска, ...
1    С 01 июня по 31 июля на осенний семестр и с 01...
2    Если вы потеряли магнитную карту (пропуск, про...
3    Для того чтобы заменить занятия по физкультуре...
4    Согласно представленному тексту, студенческий ...
dtype: object

In [32]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_pro_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.760 Recall: 0.772 F1 score: 0.764'

#### YandexGPT 3

In [30]:
yagpt_answers = []
for index, row in test_dataset_qda.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question']))
yagpt_answers = pd.Series(yagpt_answers)
yagpt_answers.head(5)

0    Вам нужно подать заявление на восстановление м...
1                                     Ответ не найден.
2    Необходимо подать заявление на восстановление ...
3    Для оформления посещения частного фитнес-клуба...
4    В течение 5 рабочих дней со дня издания приказ...
dtype: object

In [31]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(yagpt_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.678 Recall: 0.686 F1 score: 0.679'

#### `OpenChat-3.5-0106`, `saiga_llama3_8b`

In [21]:
openchat_answers = []
for index, row in test_dataset_qda.iterrows():
    openchat_answers.append(get_answer(openchat_chain, row['question']))
openchat_answers = pd.Series(openchat_answers)
openchat_answers.head(5)

0    К Единому деканату по адресу: ул. Семакова, д....
1    Даты подачи заявления для восстановления на оч...
2    При потере проходки необходимо подать заявлени...
3    Для замены обязательных занятий по физической ...
4    Ответ: Студенту, который переводится в другую ...
dtype: object

In [22]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(openchat_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.704 Recall: 0.723 F1 score: 0.711'

### 3. Тест нахождения ответа

In [36]:
# generated by CohereForAI/c4ai-command-r-plus
random_questions = """
Какой процент студентов поступает в университет из другой страны?
Какие факторы влияют на выбор студентами своего учебного заведения?
Какие существуют методы повышения мотивации студентов во время обучения?
Какие существуют методы борьбы с академическим плагиатом?
Как развиваются онлайн-курсы и дистанционное обучение в современном университете?
Какие существуют методы повышения академического Writing навыков у студентов?
Как студентам лучше всего управлять своим временем для достижения академического успеха?
Какие существуют методы повышения активности студентов на лекциях?
Как преподаватели могут эффективно использовать технологии в обучении?
Какие существуют программы развития критического мышления у студентов?
Как студентам лучше всего готовиться к экзаменам?
Какие факторы влияют на успеваемость студентов?
Какие существуют методы предотвращения студенческого стресса и выгорания?
Как студентам лучше всего искать и использовать академические источники для своих работ?
Какие существуют программы обучения за рубежом и как они влияют на студентов?
Какой процент студентов совмещает работу и учебу?
Какие существуют методы повышения разнообразия и инклюзивности в студенческом сообществе?
Какой процент студентов продолжает обучение в аспирантуре?
Какие факторы влияют на выбор студентами своей специальности?
Какие существуют методы повышения академического честности?
Как студентам лучше всего управлять своими финансами во время обучения?
Какие существуют программы поддержки психического здоровья студентов?
Какие существуют методы эффективного обучения в группе?
Как преподаватели могут адаптировать свое обучение для разных стилей обучения студентов?
Какие существуют программы развития межкультурной компетентности?
Как студентам лучше всего искать стажировки и возможности для развития карьеры?
Какие факторы влияют на вовлеченность студентов в учебный процесс?
Какие существуют методы повышения качества обучения в университете?
Как преподаватели могут эффективно использовать активные методы обучения?
Какие существуют программы поддержки студентов из малообеспеченных семей?
Какие существуют методы предотвращения студенческого отсева?
Как студентам лучше всего развивать свои soft skills?""".split("\n")
random_questions = pd.Series(random_questions)
random_questions = pd.DataFrame({"question": random_questions, "document": random_questions.apply(answer_sbert_312), "label": 0})
actual_questions = pd.DataFrame({"question": test_dataset_qda["question"].head(33), "document": test_dataset_qda["document"].head(33), "label": 1})
question_documents_label = pd.concat([actual_questions, random_questions], ignore_index=True)
question_documents_label.sample(5)

,question,document,label
60,Какие факторы влияют на вовлеченность студенто...,Документ подписан простой электронной подписью...,0
47,Как студентам лучше всего искать и использоват...,Для оформления академической справки ( справки...,0
43,Какие существуют программы развития критическо...,Сопровождение специализированных образовательн...,0
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,1
54,Как студентам лучше всего управлять своими фин...,Сопровождение специализированных образовательн...,0


#### GigaChat

In [58]:
gigachat_answers = []
for index, row in question_documents_label.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question'], row['document']))
gigachat_answers = pd.Series(gigachat_answers)
question_documents_label["gigachat"] = gigachat_answers.apply(lambda x: 0 if "вет не найден" in x else 1)
question_documents_label.sample(5)

AUTHENTICATION ERROR
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


,question,document,label,gigachat
39,Как студентам лучше всего управлять своим врем...,Для оформления академической справки ( справки...,0,0
17,Даты подачи заявления для восстановления на за...,Заявления на восстановление в ТюмГУ принимаютс...,1,0
37,Как развиваются онлайн-курсы и дистанционное о...,Студенты очной формы обучения оформляют справк...,0,0
46,Как студентам лучше всего искать и использоват...,Для оформления академической справки ( справки...,0,0
30,Могу ли я восстановиться на заочке?,Заявления на восстановление в ТюмГУ принимаютс...,1,1


In [60]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.gigachat, average='weighted')

(0.8336397058823529, 0.8333333333333334, 0.8332950631458095, None)

#### GigaChat Pro

In [17]:
gigachat_pro_answers = []
for index, row in question_documents_label[42:].iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question'], row['document']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)

Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


In [23]:
question_documents_label["gigachat_pro"] = gigachat_pro_answers.apply(lambda x: 0 if "вет не найден" in x else 1)
question_documents_label.sample(5)

,question,document,label,gigachat_pro
56,Какие существуют методы эффективного обучения ...,Сопровождение специализированных образовательн...,0,0
34,Какой процент студентов поступает в университе...,Документ подписан простой электронной подписью...,0,0
65,Как студентам лучше всего развивать свои soft ...,Сопровождение специализированных образовательн...,0,0
51,Какой процент студентов продолжает обучение в ...,Документ подписан простой электронной подписью...,0,0
27,Куда обращаться по поводу оплаты обучения?,"По вопросам оплаты обучения, суммы задолженнос...",1,1


In [24]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.gigachat_pro, average='weighted')

(0.8273809523809524, 0.803030303030303, 0.7992982456140351, None)

#### YandexGPT 3

In [78]:
yagpt_answers = []
for index, row in question_documents_label.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question'], row['document']))
yagpt_answers = pd.Series(yagpt_answers)
question_documents_label["yagpt"] = yagpt_answers.apply(lambda x: 0 if "не найден" in x else 1)
question_documents_label.sample(5)

,question,document,label,yagpt
57,Какие существуют программы развития межкультур...,Документ подписан простой электронной подписью...,0,1
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,1,1
26,Где получить информацию о справке в военкомат?,Вам необходимо обратиться в Отдел мобилизацион...,1,1
14,Какие доки нужны для перевода в ТюмГУ из друго...,Пакет документов для подачи заявления на перев...,1,1
38,Какие существуют методы повышения академическо...,Сопровождение специализированных образовательн...,0,1


In [82]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.yagpt, average='weighted')

(0.4678362573099415, 0.48484848484848486, 0.40634920634920635, None)

#### `OpenChat-3.5-0106`, `saiga_llama3_8b`

In [37]:
openchat_answers = []
for index, row in question_documents_label.iterrows():
    openchat_answers.append(get_answer(openchat_chain, row['question'], row['document']))
openchat_answers = pd.Series(openchat_answers)
question_documents_label["openchat"] = openchat_answers.apply(lambda x: 0 if "не найден" in x else 1)
question_documents_label.sample(5)

,question,document,label,openchat
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,1,1
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,1,1
14,Какие доки нужны для перевода в ТюмГУ из друго...,Пакет документов для подачи заявления на перев...,1,1
38,Как развиваются онлайн-курсы и дистанционное о...,Студенты очной формы обучения оформляют справк...,0,0
29,Как повторно защитить диплом?,Повторно пройти государственную итоговую аттес...,1,0


In [38]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.openchat, average='weighted')

(0.7727272727272727, 0.7424242424242424, 0.7350649350649352, None)